<a href="https://www.kaggle.com/code/joeroberts/inference-bert-transfer-learning?scriptVersionId=112948823" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# used this tutorial to help. 
# https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert
# https://www.tensorflow.org/text/tutorials/classify_text_with_bert
# https://www.tensorflow.org/tfmodels/nlp/fine_tune_bert#import_libraries

import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler



import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow import keras
from tensorflow.keras import layers

from transformers import BertTokenizer, AutoTokenizer, AutoModelForSequenceClassification, TFBertModel, BertConfig

from tqdm import tqdm
# plots and images
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image

#sklearn processing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import os

# import wandb


# #configs
# max_tokens = 20000
# embed_dim  = 300
# num_heads  = 2
# dense_dim  = 32


developing = True

BATCH_SIZE = 12
BUFFER_SIZE = 3200
SEQ_LEN = 1536
AUTO = tf.data.AUTOTUNE

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv
/kaggle/input/bertbasecased/config.json
/kaggle/input/bertbasecased/tokenizer.json
/kaggle/input/bertbasecased/tokenizer_config.json
/kaggle/input/bertbasecased/pytorch_model.bin
/kaggle/input/bertbasecased/vocab.txt
/kaggle/input/inference/bert_transfer_model.keras
/kaggle/input/glove/glove.6B.300d.txt


In [3]:
sample_submission_raw = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv')
train_data_raw = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
test_from_comp = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')


train, test = train_test_split(train_data_raw, test_size=0.2, random_state=21) #20% for test
train, val = train_test_split(train, test_size=0.1, random_state=21) # 10% for validation


targets=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

train_y = train[targets]
val_y = val[targets]
test_y = test[targets]

train_x = train['full_text']
val_x = val['full_text']
test_x = test['full_text']
test_data_comp = test_from_comp['full_text']


In [4]:
tokenizer = BertTokenizer.from_pretrained("/kaggle/input/bertbasecased/")

def get_ids_mask(inputs):
    input_ids_part_one = []
    attention_mask_part_one = []
    
    input_ids_part_two = []
    attention_mask_part_two = []
    
    input_ids_part_three = []
    attention_mask_part_three = []
    

    for x in tqdm(inputs):
        tokens = tokenizer(x, padding="max_length", truncation=True, max_length=SEQ_LEN, return_tensors="np")
        ids = tokens["input_ids"]
        mask = tokens["attention_mask"]
        
        input_ids_part_one.append(ids[0][:512])
        attention_mask_part_one.append(mask[0][:512])
        
        input_ids_part_two.append(ids[0][512:1024])
        attention_mask_part_two.append(mask[0][512:1024])
        
        input_ids_part_three.append(ids[0][1024:1536])
        attention_mask_part_three.append(mask[0][1024:1536])
        
    input_ids_part_one = np.array(input_ids_part_one).squeeze()
    attention_mask_part_one = np.array(attention_mask_part_one).squeeze()
    
    input_ids_part_two = np.array(input_ids_part_two).squeeze()
    attention_mask_part_two = np.array(attention_mask_part_two).squeeze()
    
    input_ids_part_three = np.array(input_ids_part_three).squeeze()
    attention_mask_part_three = np.array(attention_mask_part_three).squeeze()
    
    return input_ids_part_one, attention_mask_part_one, input_ids_part_two, attention_mask_part_two, input_ids_part_three, attention_mask_part_three


train_input_ids_part_one, train_attention_mask_part_one, train_input_ids_part_two, train_attention_mask_part_two, train_input_ids_part_three, train_attention_mask_part_three = get_ids_mask(train_x)
val_input_ids_part_one, val_attention_mask_part_one, val_input_ids_part_two, val_attention_mask_part_two, val_input_ids_part_three, val_attention_mask_part_three = get_ids_mask(val_x)
test_input_ids_part_one, test_attention_mask_part_one, test_input_ids_part_two, test_attention_mask_part_two, test_input_ids_part_three, test_attention_mask_part_three = get_ids_mask(test_x)
test_data_comp_input_ids_part_one, test_data_comp_attention_mask_part_one, test_data_comp_input_ids_part_two, test_data_comp_attention_mask_part_two, test_data_comp_input_ids_part_three, test_data_comp_attention_mask_part_three = get_ids_mask(test_data_comp)



100%|██████████| 3/3 [00:00<00:00, 88.40it/s]


In [7]:
class MeanPool(keras.layers.Layer):
    def call(self, x, mask=None):
        broad_mask = tf.cast(tf.expand_dims(mask, -1), "float32")
        x = tf.math.reduce_sum( x * broad_mask, axis=1)
        x = x / tf.math.maximum(tf.reduce_sum(broad_mask, axis=1), tf.constant([1e-9]))
        return x 


In [10]:
model = keras.models.load_model(
    "/kaggle/input/inference/bert_transfer_model.keras",                 
   custom_objects={
        "MeanPool": MeanPool

        })

In [11]:
model.evaluate(
    {
            'tokens_one': test_input_ids_part_one,
            'attention_mask_one': test_attention_mask_part_one,
            
            'tokens_two': test_input_ids_part_two,
            'attention_mask_two': test_attention_mask_part_two,
            
            'tokens_three': test_input_ids_part_three,
            'attention_mask_three': test_attention_mask_part_three,
        },
    test_y , 1)

2022-12-04 22:21:26.069589: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


783/783 [==============================] - 69s 81ms/step - loss: 0.2577 - root_mean_squared_error: 0.5077


[0.2577137351036072, 0.5076553225517273]

In [12]:
test_predictions = model.predict(x={
            'tokens_one': test_data_comp_input_ids_part_one,
            'attention_mask_one': test_data_comp_attention_mask_part_one,
            
            'tokens_two': test_data_comp_input_ids_part_two,
            'attention_mask_two': test_data_comp_attention_mask_part_two,
            
            'tokens_three': test_data_comp_input_ids_part_three,
            'attention_mask_three': test_data_comp_attention_mask_part_three,
        })


test_predictions = pd.DataFrame(test_predictions)
test_from_comp['cohesion'] = test_predictions[0]
test_from_comp['syntax'] = test_predictions[1]
test_from_comp['vocabulary'] = test_predictions[2]
test_from_comp['phraseology'] = test_predictions[3]
test_from_comp['grammar'] = test_predictions[4]
test_from_comp['conventions'] = test_predictions[5]
test_from_comp

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,when a person has no experience on a job their...,2.930954,2.974277,3.140543,3.016876,2.738055,2.684046
1,000BAD50D026,Do you think students would benefit from being...,2.895386,2.713362,2.950342,2.719229,2.628312,2.732231
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",3.526787,3.520761,3.643239,3.608127,3.561570,3.626666


In [13]:
res = test_from_comp.drop("full_text",axis=1)
res.to_csv("/kaggle/working/submission.csv",index=False)